In [1]:
import math, gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from onemapsg import OneMapClient
from haversine import haversine, Unit
from geopy.geocoders import Nominatim
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")

# import
malls = pd.read_csv(r'C:\Users\xBaka\Notebooks\School\Project\used\malls.csv')
mrt_lrt = pd.read_csv(r'C:\Users\xBaka\Notebooks\School\Project\used\mrt_lrt.csv')
markets = pd.read_csv(r'C:\Users\xBaka\Notebooks\School\Project\used\markets.csv')
hawkers = pd.read_csv(r'C:\Users\xBaka\Notebooks\School\Project\used\hawkers.csv')
addresses = pd.read_csv(r'C:\Users\xBaka\Notebooks\School\Project\used\unique_addresses.csv')
cpi = pd.read_csv(r'C:\Users\xBaka\Notebooks\School\Project\used\rpi-quarterly.csv')
data = pd.read_csv(r'D:\School stuff\Project/resale-1990-onwards.csv')

In [2]:
# Month
"""
Convert month string into Python datetime format.
Subtract 30 days from month to obtain correct month
"""
data['month'] = pd.to_datetime(data['month'])
data['month'] = data['month'] - timedelta(1)
data = data[(data['month']>="2015-01-01") & (data['month']<"2021-01-01")]
data = data.sort_values(by='month')
data = data.reset_index(drop=True)

In [3]:
# Quarters
"""
Extract Quarters from months
"""
def quarters(x):
    if x.month in [1,2,3]:
        return "Q1"
    elif x.month in [4,5,6]:
        return "Q2"
    elif x.month in [7,8,9]:
        return "Q3"    
    elif x.month in [10,11,12]:
        return "Q4"

data['quarter'] = data['month'].apply(quarters)

In [4]:
# Flat Type
"""
Merge flat_type and flat_model into a single feature.
Transform terrace and adjoined flat into flat_type classes
"""
data['flat_type'] = data['flat_type'].str.replace("MULTI GENERATION", "MULTI-GENERATION")
data['flat_type'] = np.where(data['flat_model']=='TERRACE', data['flat_model'], data['flat_type'])
data['flat_type'] = np.where(data['flat_model']=='ADJOINED FLAT', 'JUMBO', data['flat_type'])

In [5]:
# Street Name
"""
Convert towns to reflect street names for some towns
"""
data.loc[data[data['town'].str.contains('KALLANG')].index, 'town'] = 'KALLANG'
data.loc[data[data['street_name'].str.contains('PAYA LEBAR')].index, 'town'] = 'PAYA LEBAR'
data.loc[data[data['street_name'].str.contains('HOUGANG')].index, 'town'] = 'HOUGANG'
data.loc[data[data['street_name'].str.contains('ROCHOR')].index, 'town'] = 'ROCHOR'

In [6]:
# Address
"""
Combine block and street name to obtain full address
"""
data['address'] = data['block'] + " " + data['street_name']

In [7]:
# Storey_range
"""
Discretize storey_range into fewer
"""
storey_map = {
     '01 TO 03': 'Low', '01 TO 05': 'Low', '04 TO 06': 'Low', 
     '07 TO 09': 'Middle', '06 TO 10': 'Middle',
     '10 TO 12': 'High', '11 TO 15': 'High',
     '13 TO 15': 'High', '16 TO 18': 'High',
     '16 TO 20': 'High', '19 TO 21': 'High',
     '21 TO 25': 'High', '22 TO 24': 'High',
     '25 TO 27': 'High', '28 TO 30': 'High',
     '26 TO 30': 'High', '31 TO 33': 'High',
     '31 TO 35': 'High', '34 TO 36': 'High',
     '36 TO 40': 'High', '37 TO 39': 'High',
     '40 TO 42': 'High', '43 TO 45': 'High',
     '46 TO 48': 'High', '49 TO 51': 'High'
 }

data['storey_range'] = data['storey_range'].map(storey_map)

In [8]:
# Remaining_lease
data['remaining_lease'] = data['remaining_lease'].fillna(0)

def remaining_lease(row):
    """Take in row index and outputs remaining_lease"""
    lease_year = data['lease_commence_date'][row]
    transaction_year = data['month'][row].year
    if data['remaining_lease'][row] == 0:
        remaining_lease = int(99 - (transaction_year - lease_year))
        # some listing date before lease commence date,
        # remaining_lease was showing more than 99 years
        if remaining_lease > 99:
            return 99 
        else:
            return remaining_lease
    else:
        remaining_lease = int(data['remaining_lease'][row].split(' ')[0])
        if remaining_lease > 99:
            return 99 
        else:
            return remaining_lease

data['remaining_lease'] = data.reset_index()['index'].apply(remaining_lease)

In [9]:
# Reverse Geocoding
"""
Initialize dataframe to capture only unique address to reduce
redundancy. Further increase efficiency by pre-computing available addresses
and only computing address that are not in addresses dataframe
"""
unique_address = pd.DataFrame({'address': data['address'].unique()})
## Using OneMap API
Client = OneMapClient(email='syalabi.seet@hotmail.com', password='NnCWpDKAJt5j2wJ')
def latitude(row):
    x = unique_address['address'][row]
    if x not in unique_address['address'].to_list():
        try:
            return (Client.search(x)['results']).pop()['LATITUDE']
        except:
            return np.nan
    else:
        return addresses['latitude'][row]

def longitude(row):
    x = unique_address['address'][row]
    if x not in unique_address['address'].to_list():
        try:
            return (Client.search(x)['results']).pop()['LONGITUDE']
        except:
            return np.nan
    else:
        return addresses['longitude'][row]

def full_address(row):
    x = unique_address['address'][row]
    if x not in unique_address['address'].to_list():
        try:
            return (Client.search(x)['results']).pop()['ADDRESS']
        except:
            return np.nan
    else:
        return addresses['full_address'][row]

def postal(row):
    x = unique_address['address'][row]
    if x not in unique_address['address'].to_list():
        try:
            return (Client.search(x)['results']).pop()['POSTAL']
        except:
            return np.nan
    else:
        return addresses['postal_code'][row]

unique_address['full_address'] = unique_address.reset_index()['index'].apply(full_address)        
unique_address['postal_code'] = unique_address.reset_index()['index'].apply(postal)
unique_address['latitude'] = unique_address.reset_index()['index'].apply(latitude)
unique_address['longitude'] = unique_address.reset_index()['index'].apply(longitude)

unique_address = unique_address.dropna()
unique_address['coordinates'] = list(zip(unique_address['latitude'].astype(float), unique_address['longitude'].astype(float)))

In [10]:
# Distances from amenities
"""
Calculate haversine distances using coordinates.
All amenities data are pre-computed, only the distances
has to be computed on-the-fly.
"""
class haversine_func():
    def station_location(p1):
        nearest_y = 1000
        p3 = 0
        distances = []
        for i, p2 in zip(mrt_lrt['station_name'], mrt_lrt['coordinates']):
            y = haversine(p1,p2) # distance
            distances.append(y)
            if y < nearest_y:
                nearest_y = y
                p3 = i
        return str(p3)
    
    def station_distance(p1):
        nearest_y = 1000
        p3 = 0
        distances = []
        for i, p2 in zip(mrt_lrt['station_name'], mrt_lrt['coordinates']):
            y = haversine(p1,p2) # distance
            distances.append(y)
            if y < nearest_y:
                nearest_y = y
                p3 = i
        return float(nearest_y)

    def mall_location(p1):
        nearest_y = 1000
        p3 = 0
        distances = []
        for i, p2 in zip(malls['Shopping_Malls'], malls['coordinates']):
            y = haversine(p1,p2) # distance
            distances.append(y)
            if y < nearest_y:
                nearest_y = y
                p3 = i
        return str(p3)
    
    def mall_distance(p1):
        nearest_y = 1000
        p3 = 0
        distances = []
        for i, p2 in zip(malls['Shopping_Malls'], malls['coordinates']):
            y = haversine(p1,p2) # distance
            distances.append(y)
            if y < nearest_y:
                nearest_y = y
                p3 = i
        return float(nearest_y)

    def market_location(p1):
        nearest_y = 1000
        p3 = 0
        distances = []
        for i, p2 in zip(markets['licensee_name'], markets['coordinates']):
            y = haversine(p1,p2) # distance
            distances.append(y)
            if y < nearest_y:
                nearest_y = y
                p3 = i
        return str(p3)
    
    def market_distance(p1):
        nearest_y = 1000
        p3 = 0
        distances = []
        for i, p2 in zip(markets['licensee_name'], markets['coordinates']):
            y = haversine(p1,p2) # distance
            distances.append(y)
            if y < nearest_y:
                nearest_y = y
                p3 = i
        return float(nearest_y)

    def hawker_location(p1):
        nearest_y = 1000
        p3 = 0
        distances = []
        for i, p2 in zip(hawkers['name_of_centre'], hawkers['coordinates']):
            y = haversine(p1,p2) # distance
            distances.append(y)
            if y < nearest_y:
                nearest_y = y
                p3 = i
        return str(p3)
    
    def hawker_distance(p1):
        nearest_y = 1000
        p3 = 0
        distances = []
        for i, p2 in zip(hawkers['name_of_centre'], hawkers['coordinates']):
            y = haversine(p1,p2) # distance
            distances.append(y)
            if y < nearest_y:
                nearest_y = y
                p3 = i
        return float(nearest_y)

    def cbd_distance(p1):
        # Orchard MRT coordinates
        p2 = (1.3097253474694304, 103.83155367934316)
        return haversine(p1, p2)

malls['coordinates'] = list(zip(malls['latitude'].astype(float), malls['longitude'].astype(float)))
mrt_lrt['coordinates'] = list(zip(mrt_lrt['latitude'].astype(float), mrt_lrt['longitude'].astype(float)))
markets['coordinates'] = list(zip(markets['latitude'].astype(float), markets['longitude'].astype(float)))
hawkers['coordinates'] = list(zip(hawkers['latitude'].astype(float), hawkers['longitude'].astype(float)))

unique_address = unique_address[unique_address['latitude'].notnull()]
unique_address['nearest_MRT'] = unique_address['coordinates'].apply(haversine_func.station_location)
unique_address['MRT_distance'] = unique_address['coordinates'].apply(haversine_func.station_distance)
unique_address['nearest_Mall'] = unique_address['coordinates'].apply(haversine_func.mall_location)
unique_address['Mall_distance'] = unique_address['coordinates'].apply(haversine_func.mall_distance)
unique_address['nearest_Market'] = unique_address['coordinates'].apply(haversine_func.market_location)
unique_address['Market_distance'] = unique_address['coordinates'].apply(haversine_func.market_distance)
unique_address['nearest_Hawker'] = unique_address['coordinates'].apply(haversine_func.hawker_location)
unique_address['Hawker_distance'] = unique_address['coordinates'].apply(haversine_func.hawker_distance)
unique_address['CBD_distance'] = unique_address['coordinates'].apply(haversine_func.cbd_distance)

data = pd.merge(left=data, right=unique_address, left_on='address', right_on='address').drop(columns=['full_address'])

In [11]:
# Town
"""
Convert town into postal district codes using postal codes
"""
def postal_map(x):
    if x in ['01', '02', '03', '04', '05', '06']:
        return '01'
    elif x in ['07', '85', '80']:
        return '02'
    elif x in ['14', '15', '16']:
        return '03'
    elif x in ['90', '91', '10']:
        return '04'
    elif x in ['11', '12', '13']:
        return '05'
    elif x in ['17']:
        return '06'
    elif x in ['18', '19']:
        return '07'
    elif x in ['20', '21']:
        return '08'
    elif x in ['22', '23']:
        return '09'
    elif x in ['24', '25', '26', '27']:
        return '10'
    elif x in ['28', '29', '30']:
        return '11'
    elif x in ['31', '32', '33']:
        return '12'
    elif x in ['34', '35', '36', '37']:
        return '13'
    elif x in ['38', '39', '40', '41']:
        return '14'
    elif x in ['42', '43', '44', '45']:
        return '15'
    elif x in ['46', '47', '48']:
        return '16'
    elif x in ['49', '50', '81']:
        return '17'
    elif x in ['51', '52']:
        return '18'
    elif x in ['53', '54', '55', '82']:
        return '19'
    elif x in ['56', '57']:
        return '20'
    elif x in ['58', '59']:
        return '21'
    elif x in ['60', '61', '62', '63', '64']:
        return '22'
    elif x in ['65', '66', '67', '68']:
        return '23'
    elif x in ['69', '70', '71']:
        return '24'
    elif x in ['72', '73']:
        return '25'
    elif x in ['77', '78']:
        return '26'
    elif x in ['75', '76']:
        return '27'
    elif x in ['79', '79']:
        return '28'
    else:
        return np.nan

data['town'] = data['postal_code'].str[0:2].apply(postal_map)
data = data.rename(columns={'town': 'district'})
data = data[data['district'].notnull()].reset_index(drop=True)

In [12]:
# Deflation
"""
Deflates historical resale price using RPI based on 2019 rates.
NaN will appear for months that does not have CPI index in CPI csv
i.e January 2021 or later
"""
def deflation_fn(row):
    while True:
        try:
            x_i = data['resale_price'][row]
            c_o = 100.0
            c_i = cpi[
                (cpi['year']==data['month'][row].year) &
                (cpi['quarter']==data['quarter'][row])
            ]['index'].item()
            return x_i * (c_o/c_i)
        except ValueError:
            return np.nan

cpi['year'] = cpi['quarter'].str.split('-', expand=True)[0].astype(int)
cpi['quarter'] = cpi['quarter'].str.split('-', expand=True)[1]
cpi = cpi[['year', 'quarter', 'index']]

data['resale_price'] = data['resale_price'].reset_index()['index'].apply(deflation_fn)

In [13]:
# Feature Selection
"""
Remove features with high cardinality
"""
data = data.drop(
    columns=[
        'street_name', 'block', 'address', 'postal_code', 
        'nearest_MRT', 'nearest_Hawker', 'nearest_Market', 
        'nearest_Mall', 'flat_model', 'latitude', 
        'longitude', 'lease_commence_date', 'quarter',
        'coordinates'
    ],
    axis=1
)

In [14]:
# Outlier Detection
"""
Remove outliers from numerical features.
Feature limits were determined from scatterplot.
"""
outliers = []
num_features = ['floor_area_sqm', 'remaining_lease', 'MRT_distance', 'Mall_distance', 'Hawker_distance', 'CBD_distance', 'resale_price']
for i, feature in enumerate(num_features):
    
    df = data[['month', feature]]
    if feature=='floor_area_sqm':
        df['outlier'] = df[feature]>200
    elif feature=='MRT_distance':
        df['outlier'] = df[feature]>2.5
    elif feature=='Mall_distance':
        df['outlier'] = df[feature]>2.5
    elif feature=='Hawker_distance':
        df['outlier'] = df[feature]>7
    elif feature=='CBD_distance':
        df['outlier'] = df[feature]>17.5
    else:
        df['outlier'] = False

    for outlier in df[df['outlier']==True].index:
        outliers.append(outlier)

outliers = set(outliers)
data = data[~data.index.isin(outliers)]

del df

In [15]:
# Onehot-Encode
X = data.drop(columns=['resale_price'], axis=1)
y = data['resale_price']

features = ['flat_type', 'storey_range', 'district']
for feature in features:
    X = pd.concat([X, pd.get_dummies(X[feature], prefix=feature)], axis=1)
X = X.drop(columns=features, axis=1)

data = pd.concat([X, pd.DataFrame({'target': y})], axis=1).reset_index(drop=True)

In [16]:
data

,month,floor_area_sqm,remaining_lease,MRT_distance,Mall_distance,Market_distance,Hawker_distance,CBD_distance,flat_type_1 ROOM,flat_type_2 ROOM,...,district_18,district_19,district_20,district_21,district_22,district_23,district_25,district_27,district_28,target
0,2015-01-31,88.0,84,0.994266,0.808769,0.047046,0.185346,6.376045,0,0,...,0,0,1,0,0,0,0,0,0,259120.206490
1,2015-01-31,86.0,84,0.994266,0.808769,0.047046,0.185346,6.376045,0,0,...,0,0,1,0,0,0,0,0,0,243362.831858
2,2015-04-30,86.0,84,0.994266,0.808769,0.047046,0.185346,6.376045,0,0,...,0,0,1,0,0,0,0,0,0,243703.703704
3,2015-07-31,88.0,83,0.994266,0.808769,0.047046,0.185346,6.376045,0,0,...,0,0,1,0,0,0,0,0,0,256315.007429
4,2015-09-30,88.0,83,0.994266,0.808769,0.047046,0.185346,6.376045,0,0,...,0,0,1,0,0,0,0,0,0,265973.254086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122846,2020-12-31,83.0,95,1.006174,1.010061,0.380619,1.659092,13.853965,0,0,...,0,0,0,0,0,0,0,1,0,419985.517741
122847,2020-12-31,42.0,53,1.607296,0.696057,0.552656,2.500426,12.342841,0,1,...,0,0,0,0,0,0,0,1,0,166545.981173
122848,2020-12-31,93.0,95,1.067377,1.070907,0.435352,1.715333,13.891424,0,0,...,0,0,0,0,0,0,0,1,0,347574.221579
122849,2020-12-31,93.0,95,1.067377,1.070907,0.435352,1.715333,13.891424,0,0,...,0,0,0,0,0,0,0,1,0,347574.221579
